In [ ]:
#word frequency distribution
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt_tab")

def word_frequency(text):
  words=word_tokenize(text.lower())
  return Counter(words)

text="NLP is amazing, NLP makes machines understand Language"
print(word_frequency(text))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Counter({'nlp': 2, 'is': 1, 'amazing': 1, ',': 1, 'makes': 1, 'machines': 1, 'understand': 1, 'language': 1})


# BAG of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus=["NLP is fun and amazing",
        "Machines understand NLP and Text",
        "Text processing is a part of NLP"
        ]


In [ ]:
Vectorizer=CountVectorizer()
x=Vectorizer.fit_transform(corpus)
print(Vectorizer.get_feature_names_out())
print(x.toarray())

['amazing' 'and' 'fun' 'is' 'machines' 'nlp' 'of' 'part' 'processing'
 'text' 'understand']
[[1 1 1 1 0 1 0 0 0 0 0]
 [0 1 0 0 1 1 0 0 0 1 1]
 [0 0 0 1 0 1 1 1 1 1 0]]


# TF-IDF

ou calculated:
TF = 1/5 = 0.2

IDF = log(3/2) ≈ 0.176

So, TF × IDF = 0.2 × 0.176 = 0.0352 ❌

But the model gave 0.534 — why?

I have upload all the details about the math intution behind tf-idf go and check there



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
y=vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(y.toarray())

['amazing' 'and' 'fun' 'is' 'machines' 'nlp' 'of' 'part' 'processing'
 'text' 'understand']
[[0.53409337 0.40619178 0.53409337 0.40619178 0.         0.31544415
  0.         0.         0.         0.         0.        ]
 [0.         0.40619178 0.         0.         0.53409337 0.31544415
  0.         0.         0.         0.40619178 0.53409337]
 [0.         0.         0.         0.35829137 0.         0.27824521
  0.4711101  0.4711101  0.4711101  0.35829137 0.        ]]


In [ ]:
import numpy as np
import math

tokenize_docs=[docs.lower().split() for docs in corpus]
tokenize_docs

[['nlp', 'is', 'fun', 'and', 'amazing'],
 ['machines', 'understand', 'nlp', 'and', 'text'],
 ['text', 'processing', 'is', 'a', 'part', 'of', 'nlp']]

In [ ]:
vocab=sorted(set(word for doc in tokenize_docs for word in doc))
vocab_index={word:idx for idx,word in enumerate(vocab)}
print(vocab)
print(vocab_index)

['a', 'amazing', 'and', 'fun', 'is', 'machines', 'nlp', 'of', 'part', 'processing', 'text', 'understand']
{'a': 0, 'amazing': 1, 'and': 2, 'fun': 3, 'is': 4, 'machines': 5, 'nlp': 6, 'of': 7, 'part': 8, 'processing': 9, 'text': 10, 'understand': 11}


In [ ]:
def compute_tf(doc_tokens):
  tf_vector=np.zeros(len(vocab))
  word_counts=Counter(doc_tokens)
  for word,count in word_counts.items():
    tf_vector[vocab_index[word]]=count/len(doc_tokens)
  return tf_vector

tf_matrix=np.array([compute_tf(doc) for doc in tokenize_docs])
tf_matrix

array([[0.        , 0.2       , 0.2       , 0.2       , 0.2       ,
        0.        , 0.2       , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.2       , 0.        , 0.        ,
        0.2       , 0.2       , 0.        , 0.        , 0.        ,
        0.2       , 0.2       ],
       [0.14285714, 0.        , 0.        , 0.        , 0.14285714,
        0.        , 0.14285714, 0.14285714, 0.14285714, 0.14285714,
        0.14285714, 0.        ]])

In [ ]:
def compute_idf():
  N=len(tokenize_docs)
  idf=np.zeros(len(vocab))
  for word,idx in vocab_index.items():
    df=sum(1 for doc in tokenize_docs if word in doc)
    idf[idx]=math.log((N+1)/(df+1))+1
  return idf

idf_vector=compute_idf()
print(idf_vector)



[1.69314718 1.69314718 1.28768207 1.69314718 1.28768207 1.69314718
 1.         1.69314718 1.69314718 1.69314718 1.28768207 1.69314718]


In [ ]:
tfidf_matrix=tf_matrix*idf_vector
print(tfidf_matrix)

[[0.         0.33862944 0.25753641 0.33862944 0.25753641 0.
  0.2        0.         0.         0.         0.         0.        ]
 [0.         0.         0.25753641 0.         0.         0.33862944
  0.2        0.         0.         0.         0.25753641 0.33862944]
 [0.24187817 0.         0.         0.         0.18395458 0.
  0.14285714 0.24187817 0.24187817 0.24187817 0.18395458 0.        ]]


# NLP Task-Feature Extraction


In [ ]:
#sorted according to weightage
import numpy as np
feature_array=np.array(vectorizer.get_feature_names_out())
importance=np.argsort(x.toarray()).flatten()[::-1]  #First always convert into 1d vector
keywords=feature_array[importance[:10]]  #top 10 imporatnat feature

print(keywords)

['text' 'processing' 'part' 'nlp' 'of' 'is' 'understand' 'machines' 'fun'
 'and']


In [ ]:
top=3
for i,row in enumerate(tfidf_matrix):
  top_indices=row.argsort()[::-1][:top]
  keywords=[vocab[idx] for idx in top_indices]
  print(keywords)

['fun', 'amazing', 'and']
['understand', 'machines', 'and']
['part', 'processing', 'a']
